In [1]:
# Import libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

# Load datasets
customers = pd.read_csv('C:\\Users\\KIIT\\Downloads\\Customers.csv')
products = pd.read_csv('C:\\Users\\KIIT\\Downloads\\Products.csv')
transactions = pd.read_csv('C:\\Users\\KIIT\\Downloads\\Transactions.csv')

# Merge datasets
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

# Feature Engineering: Create customer profiles
customer_features = merged_data.groupby('CustomerID').agg(
    TotalSpending=('TotalValue', 'sum'),
    TransactionCount=('TransactionID', 'count'),
    AvgTransactionValue=('TotalValue', 'mean')
).reset_index()

# Adding customer region as a feature
region_dummies = pd.get_dummies(customers.set_index('CustomerID')['Region'], prefix='Region')
customer_features = customer_features.merge(region_dummies, left_on='CustomerID', right_index=True)

# Normalize features
scaler = StandardScaler()
normalized_features = scaler.fit_transform(customer_features.drop(columns=['CustomerID']))

# Calculate similarity matrix
similarity_matrix = cosine_similarity(normalized_features)

# Create recommendations for customers C0001–C0020
customer_ids = customer_features['CustomerID']
recommendations = {}

for i in range(20):  # First 20 customers (C0001 to C0020)
    customer_index = i
    similarity_scores = similarity_matrix[customer_index]
    similar_customers = sorted(
        list(zip(customer_ids, similarity_scores)),
        key=lambda x: x[1],
        reverse=True
    )[1:4]  # Top 3 similar customers (excluding self)
    recommendations[customer_ids.iloc[customer_index]] = similar_customers

# Save recommendations to CSV
recommendations_df = pd.DataFrame([
    {'cust_id': k, 'lookalikes': [(x[0], round(x[1], 3)) for x in v]}
    for k, v in recommendations.items()
])
recommendations_df.to_csv('Sarthak_Prusty_Lookalike.csv', index=False)

print("Lookalike recommendations saved to Sarthak_Prusty_Lookalike.csv")


Lookalike recommendations saved to Sarthak_Prusty_Lookalike.csv
